In [6]:
import pandas as pd
from datetime import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
from pandas.plotting import autocorrelation_plot

In [7]:
# Haal het dataframe op
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

# kleinverbruikgegevens gegevens inlezen
df_pc4 = pd.read_hdf('pc4_verbruiksdata.h5')

FileNotFoundError: File pc4_verbruiksdata.h5 does not exist